## Observations and Insights 

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as sts
from scipy.stats import sem

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
study_data = pd.merge(mouse_metadata,study_results, on="Mouse ID", how="inner")

# Display the data table for preview
study_data

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [3]:
# Checking the number of mice.
total_mice = study_data["Mouse ID"].nunique()

print(f"In the study a total number of {total_mice} mice are included.")

In the study a total number of 249 mice are included.


In [4]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 

duplicate = pd.DataFrame(study_results.duplicated(subset=["Mouse ID", "Timepoint"]))

duplicate = pd.DataFrame(duplicate.loc[duplicate[0] == True], columns=["Mouse ID"])

duplicate["Mouse ID"] = [study_results.iloc[duplicate.index[x],0] for x in range(len(duplicate))]

duplicate

,Mouse ID
137,g989
360,g989
681,g989
869,g989
1111,g989


In [5]:
# Optional: Get all the data for the duplicate mouse ID. 

difference = pd.DataFrame([study_results.iloc[duplicate.index[x],:] for x in range(len(duplicate))])

difference

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
137,g989,0,45.000000,0
360,g989,5,47.570392,0
681,g989,10,49.880528,0
869,g989,15,53.442020,0
1111,g989,20,54.657650,1


In [6]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.

clean_data = study_data.drop_duplicates(subset=["Mouse ID", "Timepoint"], keep="first")

clean_data

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [7]:
# Checking the number of mice in the clean DataFrame.

clean_total_mice = clean_data["Mouse ID"].nunique()

print(f"In the cleaned study data a total number of {clean_total_mice} mice are included.")

In the cleaned study data a total number of 249 mice are included.


## Summary Statistics

In [70]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method is the most straighforward, creating multiple series and putting them all together at the end.

number_regimen = clean_data["Drug Regimen"].unique()

regimen_stats_1 = pd.DataFrame()

for regimen in number_regimen:
    regimen_data = clean_data.loc[clean_data["Drug Regimen"] == regimen]
    stats = pd.Series([regimen, round(regimen_data["Tumor Volume (mm3)"].mean(),2),round(regimen_data["Tumor Volume (mm3)"].median(),2),round(regimen_data["Tumor Volume (mm3)"].var(),2),
    round(regimen_data["Tumor Volume (mm3)"].std(),2),round(regimen_data["Tumor Volume (mm3)"].sem(),2)])
    regimen_stats_1 = regimen_stats_1.append(stats, ignore_index=True)

regimen_stats_1 = regimen_stats_1.rename(columns={0:"Drug Regimen",1:"Tumor Volume MEAN",2:"Tumor Volume MED",3:"Tumor Volume VAR",4:"Tumor Volume STD",5:"Tumor Volume SEM"}).set_index(["Drug Regimen"])

regimen_stats_1

,Tumor Volume MEAN,Tumor Volume MED,Tumor Volume VAR,Tumor Volume STD,Tumor Volume SEM
Drug Regimen,,,,,
Ramicane,40.22,40.67,23.49,4.85,0.32
Capomulin,40.68,41.56,24.95,4.99,0.33
Infubinol,52.88,51.82,43.13,6.57,0.49
Placebo,54.03,52.29,61.17,7.82,0.58
Ceftamin,52.59,51.78,39.29,6.27,0.47
Stelasyn,54.23,52.43,59.45,7.71,0.57
Zoniferol,53.24,51.82,48.53,6.97,0.52
Ketapril,55.24,53.70,68.55,8.28,0.60
Propriva,52.39,50.91,43.14,6.57,0.53


In [31]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function

grouped_regimen_data = clean_data.groupby("Drug Regimen")

regimen_stats_2 = round(pd.DataFrame(grouped_regimen_data["Tumor Volume (mm3)"].mean()),2).rename(columns={"Tumor Volume (mm3)":"Tumor Volume MEAN"})
regimen_stats_2["Tumor Volume MED"] = round(grouped_regimen_data["Tumor Volume (mm3)"].median(),2)
regimen_stats_2["Tumor Volume VAR"] = round(grouped_regimen_data["Tumor Volume (mm3)"].var(),2)
regimen_stats_2["Tumor Volume STD"] = round(grouped_regimen_data["Tumor Volume (mm3)"].std(),2)
regimen_stats_2["Tumor Volume SEM"] = round(grouped_regimen_data["Tumor Volume (mm3)"].sem(),2)
regimen_stats_2

,Tumor Volume MEAN,Tumor Volume MED,Tumor Volume VAR,Tumor Volume STD,Tumor Volume SEM
Drug Regimen,,,,,
Capomulin,40.68,41.56,24.95,4.99,0.33
Ceftamin,52.59,51.78,39.29,6.27,0.47
Infubinol,52.88,51.82,43.13,6.57,0.49
Ketapril,55.24,53.70,68.55,8.28,0.60
Naftisol,54.33,52.51,66.17,8.13,0.60
Placebo,54.03,52.29,61.17,7.82,0.58
Propriva,52.39,50.91,43.14,6.57,0.53
Ramicane,40.22,40.67,23.49,4.85,0.32
Stelasyn,54.23,52.43,59.45,7.71,0.57


## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 



In [ ]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
